In [29]:
import csv
import glob
import pandas as pd
import pickle

In [30]:
path = '/Users/elizabeth/Desktop/line_broadening.nosync/line_broadening/model_search/'
file_list = glob.glob(path + 'raw_data/*.csv')

In [31]:
with open(path+'raw_data/S2.csv', newline='') as f:
    reader = csv.reader(f)
    row1 = next(reader)

print(row1)

['', 'J"', "J'", 'molecule_weight', 'm', 'd', 'dair', 'findair', 'molecule_dipole', 'polar', 'B0a', 'B0b', 'B0c', 'J', 'Jpp', 'Ka_aprox', 'Kc_aprox', 'Kapp_aprox', 'Kcpp_aprox', 'gamma_air', 'gamma_self', 'n_air', 'delta_air', 'gamma_air-err', 'gamma_self-err', 'M', 'broadness_jeanna']


In [32]:
db = {}

In [33]:
for file in file_list:
    with open(file, newline='') as f:
        count=0
        reader = csv.reader(f)
        row1 = next(reader)
        row2 = next(reader)
        i = file[86:-4]
        db[i] = pd.DataFrame(row2, index=row1)


In [34]:
db2 = {}
for molecule,data in db.items():
    weight = float(data.loc['molecule_weight'][0])
    dipole = float(data.loc['molecule_dipole'][0])
    m = int(data.loc['m'][0]) - 4
    d = float(data.loc['d'][0])
    polarisability = float(data.loc['polar'][0])
    B0a = float(data.loc['B0a'][0])
    B0b = float(data.loc['B0b'][0])
    B0c = float(data.loc['B0c'][0])
    dict = {'weight':weight, 'dipole':dipole, 'm':m, 'd':d, 'polar':polarisability, 'B0a':B0a, 'B0b':B0b, 'B0c':B0c}
    db2[molecule] = dict

In [35]:
air_weight = float(db['CS2'].loc['air_weight'][0])
air_dipole = 0
air_m = 4
air_d = float(db['CS2'].loc['dair'][0])
air_polarisability = db2['O2']['polar']*0.21 + db2['N2']['polar']*0.79
air_B0a = db2['O2']['B0a']*0.21 + db2['N2']['B0a']*0.79
air_B0b = db2['O2']['B0b']*0.21 + db2['N2']['B0b']*0.79
air_B0c = db2['O2']['B0c']*0.21 + db2['N2']['B0c']*0.79
dict = {'weight':air_weight, 'dipole':air_dipole, 'm':air_m, 'd':air_d, 'polar':air_polarisability, 'B0a':air_B0a, 'B0b':air_B0b, 'B0c':air_B0c}
db2['air'] = dict

KeyError: 'CS2'

In [359]:
He_weight = 4.002602     # pubchem NIH
He_dipole = 0
He_m = 6
He_d = 258     # Jeanna
He_polarisability = 0.208    # CCCBDB
He_B0a = 100000
He_B0b = 100000
He_B0c = 100000
dict = {'weight':He_weight, 'dipole':He_dipole, 'm':He_m, 'd':He_d, 'polar':He_polarisability, 'B0a':He_B0a, 'B0b':He_B0b, 'B0c':air_B0c}
db2['He'] = dict

In [360]:
isotope_masses = {}

isotope_masses['1H'] = 1.0078250322
isotope_masses['4He'] = 4.00260325413
isotope_masses['7Li'] = 7.0160034366
isotope_masses['9Be'] = 9.012183065
isotope_masses['11B'] = 11.00930536
isotope_masses['12C'] = 12.0000000
isotope_masses['14N'] = 14.00307400443
isotope_masses['16O'] = 15.99491461957
isotope_masses['19F'] = 18.99840316273
isotope_masses['20Ne'] = 19.9924401762	
isotope_masses['23Na'] = 22.9897692820
isotope_masses['24Mg'] = 23.985041697
isotope_masses['27Al'] = 26.98153853
isotope_masses['28Si'] = 27.97692653465
isotope_masses['31P'] = 30.97376199842
isotope_masses['32S'] = 31.9720711744
isotope_masses['35Cl'] = 34.968852682
isotope_masses['40Ar'] = 39.962383123735
isotope_masses['39K'] = 38.96370648641
isotope_masses['40Ca'] = 39.962590863
isotope_masses['45Sc'] = 44.95590828
isotope_masses['48Ti'] = 47.94794198
isotope_masses['51V'] = 50.94395704
isotope_masses['52Cr'] = 51.94050623
isotope_masses['56Fe'] = 55.93493633
isotope_masses['58Ni'] = 57.9353429
isotope_masses['75As'] = 74.92159457
isotope_masses['89Y'] = 88.9058403
isotope_masses['90Zr'] = 89.9046977
isotope_masses['139La'] = 138.9063563


with open('/Users/elizabeth/Desktop/line_broadening.nosync/line_broadening//creation_of_air_diet_files/molecules.pkl', 'rb') as f:
    molecules_made = pickle.load(f)
    
molecules_no_iso = {}
for molecule in molecules_made.keys():
    atoms = molecule.split('-')
    at2 = []
    flag = False
    for atom in atoms:
        if atom.rstrip('0123456789') not in isotope_masses:
            flag = True
        atom = atom.lstrip('0123456789.-')
        at2.append(atom)
    atoms = ''.join(at2)
    if flag:
        continue
    molecules_no_iso[atoms] = molecules_made[molecule]
    
del molecules_no_iso['CS']

for molecule, data in molecules_no_iso.items():
    weight = float(data['molecule_weight'][0])
    if 'Cl' in molecule:
        weight = weight - 34.968852682 + 35.4527
    dipole = float(data['molecule_dipole'][0])
    m = int(data['m'][0]) - 4
    d = 2*float(data['findair'][0]) - float(air_d)
    polarisability = float(data['polar'][0])
    B0a = float(data['B0a'][0])
    B0b = float(data['B0b'][0])
    B0c = float(data['B0c'][0])
    dict = {'weight':weight, 'dipole':dipole, 'm':m, 'd':d, 'polar':polarisability, 'B0a':B0a, 'B0b':B0b, 'B0c':B0c}
    if molecule in db2:
        print(molecule)
    db2[molecule] = dict

In [36]:
df = pd.DataFrame(data=db2).transpose()

In [362]:
df['dipole'].loc['CS2'] = 0    # CCCBDB
df['dipole'].loc['CH3I'] = 1.620   # CCCBDB

In [37]:
df

,B0a,B0b,B0c,d,dipole,m,polar,weight
S2,100000.0,0.29547,0.29547,447.0,0.0,4.0,6.2,63.944142


In [18]:
max(df['B0c'])

0.29547

In [19]:
min(df['B0c'])

0.29547

In [366]:
df.loc['HONO'] = df.loc['HONO_prediction']
 

In [367]:
df = df.drop('HONO_prediction', axis=0)

In [46]:
df.to_csv('molecule_parameters.csv')